# Imports

In [ ]:
import numpy as np

# Configuration

In [45]:
# directory to store intermediate and final results of the experiment
EXPERIMENT_DIR      = "/home/schindlera/experiments/ismir2020_reviews/"

AUDIO_FILENAME_STEM = "melspec_128_10seconds_2ch"

# Normalize Data

## Calculate Means from Train Partition

Calculate means over time for each Mel-band and channel

In [41]:
audio_filename_train = "%s/%s_train.npz" % (EXPERIMENT_DIR, AUDIO_FILENAME_STEM)

with np.load(audio_filename_train, allow_pickle=True) as npz:
    data  = npz["data"]

In [ ]:
    means = data.mean(axis=(0,2), keepdims=True)

In [ ]:
    stds  = data.std(axis=(0,2), keepdims=True)

## Normalize all Experiment Partitions

First, normalize the train partition, because the data is already loaded in memory

In [ ]:
global_means = np.expand_dims(global_means,0)
global_means = np.expand_dims(global_means,2)

In [ ]:
global_means = np.expand_dims(global_means,0)
global_means = np.expand_dims(global_means,2)

In [58]:
m = np.expand_dims(global_means,0)
m = np.expand_dims(m,2)

x1 = data[:2,:,:,:]
x2 = (data[:2,:,:,:] - np.repeat(m,2,axis=0))

In [10]:
for par in ["val", "test"]:

    audio_filename_par = "%s/%s_%s.npz" % (EXPERIMENT_DIR, AUDIO_FILENAME_STEM, par)
    
    with np.load(audio_filename_par, allow_pickle=True) as npz:
        data  = npz["data"]
